# Table of Contents

### 1 Scenario Generator
Generates a demand scenario for three commodities: maize, soybean and wheat, which is then fed into the simulator. Each demand trajectory consists out of a historic demand component and a scenario component. The scenario component can be created linearly, exponentially, exponentially linked to random seeding or can be predefined.
### 2 Simulator
Iterates through every year within the simulation window, is fed with demand scenario (see chapter 1), initial terminal setup and a list of investment triggers. All in all, the simulator produces a single *terminal* variable, under which all infrastructure and financial components are grouped (e.g. terminal.quays, terminal.capex, terminal.NPV etc.)
### 3 Investment Triggers Iteration
Runs the simulator (chapter 2) multiple times, while slightly changing each investment trigger. The resulting *terminal* variables are saved in a list after which the impact of each trigger alteration can be visualised.
### 4 Risk Sensitivity Iteration
Runs the simulator (chapter 2) multiple times, while slight changing the presumed risk (quantified as weighted average cost of capital (WACC)). The resulting *terminal* variables are saved in a list after which the sensitivit to risk can be visualised.
### 5 Single Simulation Run 
Runs a single simulation (chapter 2) using predefined triggers
### 6 Plots
Result visualisations

In [1]:
import plotly
import numpy as np
import pandas as pd

import plotly.tools as tls 
import plotly.plotly as py  
import plotly.graph_objs as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

import terminal_optimization.visualisation        as visualisation
import terminal_optimization.forecast             as forecast
import terminal_optimization.investment_decisions as invest
import terminal_optimization.infrastructure       as infra
import terminal_optimization.financial_analysis   as financial
import terminal_optimization.initial_terminal     as initial

# Log in to Plotly servers
plotly.tools.set_credentials_file(username='jorisneuman', api_key='zButeTrlr5xVETcyvazd')
#plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
#plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')

## 1 Traffic Generator

In [2]:
def traffic_generator(start_year, simulation_window):
    
    ###################################################################################################
    # Traffic projections on which estimate is based
    ###################################################################################################

    # Import commodities from package
    maize   = forecast.bulk_commodities(**forecast.maize_data)
    soybean = forecast.bulk_commodities(**forecast.maize_data)
    wheat   = forecast.bulk_commodities(**forecast.wheat_data)

    # Maize - Probabilistic demand
    historic_demand_maize = [1000000,1200000,1180000,1210000,1250000]
    rate_maize  = 1.02          # Avg population growth over the past 5 years
    mu_maize    = 0.01          # avg bonus rate added to base rate (% points)  - based on seasonality
    sigma_maize = 0.025         # standard deviation of bonus rate (% points)   - based on seasonality

    # Create traffic projections
    start_year = 2018
    simulation_window = 20
    traffic_projections = []
    for iterations in range(1001):
        maize.random_scenario(start_year, simulation_window, historic_demand_maize, rate_maize, mu_maize, sigma_maize)
        traffic_projections.append(maize.demand)
    
    ###################################################################################################
    # Traffic scenario based on median present quantity of all projections
    ################################################################################################### 
    
    # Calculate the net present quantity of each projection
    project_WACC = 0.09
    WACC_factor = []
    for year in range (len(historic_demand_maize)):
        WACC_factor.append(1.0)
    for year in range (simulation_window):
        WACC_factor.append(1/((1+project_WACC)**(year)))

    traffic_matrix = np.zeros(shape=(len(traffic_projections), 2))
    for i in range(len(traffic_projections)):
        traffic_matrix[i,0] = i
        present_value_volume = []
        for j in range(len(traffic_projections[i])):
            present_value_volume.append(traffic_projections[i][j]*WACC_factor[j])
        traffic_matrix[i,1] = int(np.sum(present_value_volume))

    df=pd.DataFrame(traffic_matrix, columns=['Iteration','NPQ'])

    # Select the median traffic projection in terms of present quantity
    median_iteration = df.loc[df['NPQ']==np.median(df['NPQ'])].index[0]
    maize.demand = traffic_projections[median_iteration] 
    soybean.demand = [0]*len(maize.demand)
    wheat.demand   = [0]*len(maize.demand)
    commodities = [maize, soybean, wheat]
    
    ###################################################################################################
    # Translate traffic projection into terminal calls 
    ################################################################################################### 
    
    # Import vessels from package
    handysize = forecast.vessel(**forecast.handysize_data)
    handymax  = forecast.vessel(**forecast.handymax_data)
    panamax   = forecast.vessel(**forecast.panamax_data)
    vessels = [handysize, handymax, panamax]
    
    # Calculate yearly calls
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)

    # Plot forecast
    plotdata = visualisation.scenario(traffic_projections, commodities)
    
    return vessels, commodities, plotdata, traffic_projections

## 2 Temporal Terminal Design

In [3]:
def design(terminal, vessels, commodities, start_year, simulation_window):
    
    for i in range (start_year, start_year + simulation_window):
        year = i 
        timestep = year - start_year

        ######################
        # Investment Decisions
        ######################     
        
        # Berths and cranes
        terminal.berths, terminal.cranes = invest.berth_invest_decision(terminal.berths, terminal.cranes, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)
        
        # Quay
        terminal.quays = invest.quay_invest_decision(terminal.quays, terminal.berths, year, timestep)

        # Storage
        storage_type            = 'Silos'
        terminal.storage = invest.storage_invest_decision(terminal.storage, terminal.required_storage_factor, terminal.aspired_storage_factor, storage_type, commodities, year, timestep)

        # Loading stations
        terminal.stations, trains = invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

        # Conveyors
        terminal.quay_conveyors = invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.cranes, year, timestep, operational_hours)
        terminal.hinterland_conveyors = invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, year, timestep, operational_hours)

        ######################
        # Business Logic
        ######################
        
        # Terminal throughput
        terminal.throughputs = financial.throughput_calc(terminal, commodities, vessels, trains, operational_hours, timestep, year)
        # Revenue
        terminal.revenues = financial.revenue_calc(terminal.revenues, terminal.throughputs, commodities, year, timestep)       
        # Capex
        terminal.capex = financial.capex_calc(terminal, year, timestep)
        # Labour costs
        terminal.labour = financial.labour_calc(terminal, year, timestep, operational_hours)
        # Maintenance costs
        terminal.maintenance = financial.maintenance_calc(terminal, year, timestep)
        # Energy costs
        terminal.energy = financial.energy_calc(terminal, year, operational_hours, timestep)
        # Insurance costs
        terminal.insurance = financial.insurance_calc(terminal, year, timestep)
        # Lease costs 
        terminal.lease = financial.lease_calc(terminal, year,timestep)
        # Demurrage costs
        terminal.demurrage = financial.demurrage_calc(terminal.demurrage, terminal.berths, vessels, year, timestep)
        # Residual value calculations 
        terminal.residuals = financial.residual_calc(terminal, year, timestep)
        # Profits
        terminal.profits = financial.profit_calc(terminal, simulation_window, timestep, year, start_year)
        # Opex
        terminal.opex = financial.opex_calc(terminal, year, timestep)  
        
    #WACC depreciated profits
    terminal.WACC_cashflows = financial.WACC_calc(terminal.project_WACC, terminal.profits, terminal.revenues, terminal.capex, terminal.opex, simulation_window, start_year)
    
    # Combine all cashflows
    terminal.cashflows = financial.cashflow_calc(terminal, simulation_window, start_year) 
    
    #NPV 
    terminal.NPV = financial.NPV_calc(terminal.WACC_cashflows)
            
    return terminal

## 3 Investment Triggers Iteration

In [4]:
# Simulation parameters
start_year        = 2018   # start year of simulation
simulation_window = 20     # forecast 20 years
end_year          = start_year + simulation_window - 1
operational_hours = 4680   # operational hours per year
project_WACC      = 0.09

# Performance triggers (static values)
triggers = []
triggers.append([0.20, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([0.07, 'Fraction of yearly throughput required as storage'])
triggers.append([0.15, 'Fraction of yearly throughput aspired as storage'])
triggers.append([0.20, 'Acceptable waiting train time as factor of service time'])

# Performance triggers (iteration values)
triggers_spectrum = [] 
triggers_spectrum.append([np.linspace(0.10, 1.00, 51), 'Acceptable waiting vessel time as factor of service time'])
triggers_spectrum.append([np.linspace(0.05, 0.08, 4), 'Fraction of yearly throughput required as storage'])
triggers_spectrum.append([np.linspace(0.07, 0.20, 14), 'Fraction of yearly throughput aspired as storage'])
triggers_spectrum.append([np.linspace(0.10, 1.50, 141), 'Acceptable waiting train time as factor of service time'])

# Make traffic projections
vessels, commodities, plotdata, traffic_projections = traffic_generator(start_year, simulation_window)

# Run multiple simulations, while iterating through each investment trigger
results = []

# Iterate through allowable vessel waiting times
iterations = []
for waiting_time in triggers_spectrum[0][0]:
    original_value = triggers[0][0]
    triggers[0][0] = waiting_time
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    iterations.append(terminal)
results.append(iterations)
triggers[0][0] = original_value

# Iterate through required storage factors
iterations = []
for storage_factor in triggers_spectrum[1][0]:
    original_value = triggers[1][0]
    triggers[1][0] = storage_factor
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    iterations.append(terminal)
results.append(iterations)
triggers[1][0] = original_value

# Iterate through aspired storage factors
iterations = []
for storage_factor in triggers_spectrum[2][0]:
    original_value = triggers[2][0]
    triggers[2][0] = storage_factor
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    iterations.append(terminal)
results.append(iterations)
triggers[2][0] = original_value

# Iterate through allowable train waiting times
iterations = []
for waiting_time in triggers_spectrum[3][0]:
    original_value = triggers[3][0]
    triggers[3][0] = waiting_time
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    iterations.append(terminal)
results.append(iterations)
triggers[3][0] = original_value

# Visualize scenario
fig = dict(data=plotdata[0], layout=plotdata[1])
#py.iplot(fig, filename='Traffic projections')

In [16]:
# Visualise NPV distribution as a result of allowable vessel waiting time
waiting_time_iterations = results[0]
plotdata = visualisation.NPV_distribution_vessel_waiting_times(waiting_time_iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
py.iplot(fig, filename='Trigger iteration - vessel waiting time')

In [6]:
# Visualise NPV distribution as a result of required storage factor
waiting_time_iterations = results[1]
plotdata = visualisation.NPV_distribution_required_storage(waiting_time_iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
#py.iplot(fig, filename='Trigger iteration - required storage factor')

In [7]:
# Visualise NPV distribution as a result of aspired storage factor
waiting_time_iterations = results[2]
plotdata = visualisation.NPV_distribution_aspired_storage(waiting_time_iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
#py.iplot(fig, filename='Trigger iteration - required storage factor')

In [15]:
# Visualise NPV distribution as a result of allowable train waiting time
waiting_time_iterations = results[3]
plotdata = visualisation.NPV_distribution_train_waiting_times(waiting_time_iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
#y.iplot(fig, filename='Trigger iteration - train waiting time')

## 4 Estimate Project Value
Iterating through the numerous traffic projections (in the case of the perfect foresight method, the single traffic scenario is fed into the value estimator) 

In [9]:
###############################
# Input
###############################

estimate_designs = []

project_WACC = 0.09

triggers = []
triggers.append([0.33, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([0.10, 'Fraction of yearly throughput required as storage'])
triggers.append([0.12, 'Fraction of yearly throughput aspired as storage'])
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])

###############################
# Perfect foresight method
###############################

# Create traffic scenario
#vessels, commodities, plotdata, traffic_projections = traffic_generator(start_year, simulation_window)

# Temporal terminal design (using future performance triggers)
#terminal = initial.terminal(project_WACC, triggers)
#terminal = design(terminal, vessels, commodities, start_year, simulation_window)
#estimate_designs.append(terminal)

###############################
# Current performance method
###############################

for i in range(len(traffic_projections)):
    # Redefine traffic demand
    commodities[0].demand = traffic_projections[i]
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)
    
    # Temporal terminal design (using current performance triggers)
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    estimate_designs.append(terminal)
    
###############################
# Forecast based method
###############################

#for i in range(len(traffic_projections)):
#    # Redefine traffic demand
#    commodities[0].demand = traffic_projections[i]
#    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)
    
    # Temporal terminal design (using forecast performance triggers)
#    terminal = initial.terminal(project_WACC, triggers)
#    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
#    estimate_designs.append(terminal)

###############################
# Visualization
###############################

# Visualise NPV distribution of all terminal designs
fig, NPV_estimation = visualisation.NPV_estimation(estimate_designs)
py.iplot(fig, filename='Project value estimate')

# 5 Evaluate Financial Performance

In [10]:
# Create traffic simulations
vessels, commodities, plotdata, traffic_simulations = traffic_generator(start_year, simulation_window)
evaluate_designs = []

###############################
# Perfect foresight method
###############################

# Temporal terminal design (using future performance triggers)
#terminal = initial.terminal(project_WACC, triggers)
#terminal = design(terminal, vessels, commodities, start_year, simulation_window)
#evaluate_designs.append(terminal)

###############################
# Current performance method
###############################

for i in range(len(traffic_simulations)):
    # Redefine traffic demand
    commodities[0].demand = traffic_simulations[i]
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)
    
    # Temporal terminal design (using current performance triggers)
    terminal = initial.terminal(project_WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    evaluate_designs.append(terminal)
    
###############################
# Forecast based method
###############################

#for i in range(len(traffic_projections)):
#    # Redefine traffic demand
#    commodities[0].demand = traffic_projections[i]
#    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)
    
    # Temporal terminal design (using forecast performance triggers)
#    terminal = initial.terminal(project_WACC, triggers)
#    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
#    evaluate_designs.append(terminal)

###############################
# Visualization
###############################

# Visualise NPV distribution of all terminal designs
fig = visualisation.NPV_evaluation(evaluate_designs, NPV_estimation)
py.iplot(fig, filename='Financial performance')    

# 6 Evaluate Design Method

In [11]:
fig = visualisation.method_evaluation(estimate_designs, evaluate_designs)
py.iplot(fig, filename='Method evaluation')

## 7 Single Simulation Run

In [12]:
project_WACC = 0.09

triggers = []
triggers.append([0.33, 'Acceptable waiting vessel time as factor of service time'])
triggers.append([0.10, 'Fraction of yearly throughput required as storage'])
triggers.append([0.12, 'Fraction of yearly throughput aspired as storage'])
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])

terminal = initial.terminal(project_WACC, triggers)
terminal = design(terminal, vessels, commodities, start_year, simulation_window)

# Visualise terminal discounted cashflows 
plotdata = visualisation.revenue_capex_opex(terminal)
py.iplot({'data': plotdata[0], 'layout': plotdata[1]}, filename='Cashflows')

## 8 Risk Sensitivity Iteration

In [13]:
# Iterate through project WACC
WACC_spectrum = np.linspace(0.01,0.15,15)

WACC_iterations = []
for WACC in WACC_spectrum:
    terminal = initial.terminal(WACC, triggers)
    terminal = design(terminal, vessels, commodities, start_year, simulation_window)
    WACC_iterations.append(terminal)

# Visualise NPV distribution as a result of project WACC
plotdata = visualisation.NPV_distribution_WACC(WACC_iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
py.iplot(fig, filename='Trigger iteration - vessel waiting time')

#### 6.5 Visualisation of predictive simulator

In [14]:
#visualisation.consecutive_reative_trend(commodities, simulation_window, start_year)
#visualisation.consecutive_predictive_trend(commodities, simulation_window, start_year)